In [9]:
from maxim.maxim import Maxim, Config, LoggerConfig
from maxim.logger.components.session import SessionConfig
from maxim.logger.components.trace import TraceConfig
from maxim.logger.components.session import SessionConfig
from maxim.logger.components.span import Span, SpanConfig
from maxim.logger.components.generation import GenerationConfig, GenerationError
import tiktoken
from time import time
import openai
import re
import os
from openai import OpenAI
from uuid import uuid4
from dotenv import load_dotenv




In [ ]:


# setting up maxim logger
maxim = Maxim(Config(apiKey=os.environ["MAXIM_API_KEY"]))
logger =maxim.logger(LoggerConfig(id=os.environ["MAXIM_LOG_REPO"]))
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


In [4]:
class Agent:
    def __init__(self, client: OpenAI, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.generation = None
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, span:Span, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        self.generationConfig = GenerationConfig(id=str(uuid4()), name="generation", provider="OpenAI", model="gpt-4o", model_parameters={"temperature": 0}, messages=self.messages)
        self.generation = span.generation(self.generationConfig)
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
    
        enc = tiktoken.get_encoding("o200k_base")
        llm_output = result
        messages = self.messages
        messages_string = ''.join(["role: " + entry["role"] + " content: " + entry['content'] for entry in messages])
        prompt_tokens = len(enc.encode(messages_string))
        completion_tokens = len(enc.encode(llm_output))

        print(prompt_tokens, completion_tokens)
        
        self.generation.result({
            "id": self.generation.id,
            "object": "text_completion",
            "created": int(time()),
            "model": self.generationConfig.model,
            "choices": [
                {
                    "index": 0,
                    "text": llm_output,
                    "logprobs": None,
                    "finish_reason": "stop",
                },
            ],
            "usage": {
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "total_tokens": prompt_tokens + completion_tokens,
            },
        })

        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o", messages=self.messages
        )
        return completion.choices[0].message.content


In [5]:
system_prompt = """
You operate in a structured cycle comprising **Thought**, **Action**, **Pause**, and **Observation**.  

**Thought** is used to reason about the question and decide the next steps.  

**Action** involves executing one of the available actions based on your reasoning:  
- **calculate**: Perform a mathematical operation, such as `calculate: 9 / 3 + 2`.  
- **duckduckgo_search**: Search for information, such as `duckduckgo_search: tallest mountain in the world`.  

After performing an action, return **Pause**.  

**Observation** provides the result of the action. Use this to determine the next steps.  

At the end of the cycle, output the **Answer**.  

---

**Example 1**:  

**Question**: What is the tallest mountain in the world?  
**Thought**: I need to search for information about the tallest mountain.  
**Action**: `duckduckgo_search: tallest mountain in the world`  
**Pause**

**Observation**: Mount Everest is the tallest mountain in the world, standing at 8,848 meters.  
**Thought**: I have the answer.  
**Action**: `Mount Everest`

**Answer**: The tallest mountain in the world is Mount Everest.

---

**Example 2**:  

**Question**: What is 15 divided by 3, plus 7?  
**Thought**: I need to calculate the result.  
**Action**: `calculate: 15 / 3 + 7`  
**Pause**

**Observation**: The result is 12.  
**Thought**: I have the answer.  
**Action**: `12`

**Answer**: The result of 15 divided by 3, plus 7, is 12.
""".strip()

In [6]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

def duckduckgo_search(query, max_results=2, source="web"):
    wrapper = DuckDuckGoSearchAPIWrapper(max_results=max_results)
    search = DuckDuckGoSearchResults(api_wrapper=wrapper, source=source)
    return search.invoke(query)

def calculate(operation: str) -> float:
    return eval(operation)

In [7]:
session = logger.session(SessionConfig(id="cm2sutfea001xlkhbngqzf4myfry"))
trace = session.trace(TraceConfig(id=uuid4().hex))

def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "duckduckgo_search"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        span = trace.span(SpanConfig(id=str(uuid4()), name=f"Span : {i}"))
        result = agent(span,next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            span.name = f"Span : {i} - {action}"
            chosen_tool = action[0][0]
            tool_span = span.span(SpanConfig(id=str(uuid4()), name=f"Tool Call {chosen_tool}"))
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                tool_span.event(str(uuid4()), f"Tool Call - choosen_tool:args: {chosen_tool}:{arg}",{})
                tool_span.event(str(uuid4()), f"Tool Call - result: {result_tool}",{})
                next_prompt = f"Observation: {result_tool}"
                # tool_span.end()

            else:
                next_prompt = "Observation: Tool not found"
                span.event(str(uuid4()), f"Tool not found",{})
            print(next_prompt)
            continue

        if "Answer" in result:
            span.event(str(uuid4()), f"Final Answer: {result}",{})
            
            break
        

In [8]:
loop(query="How many planets in the solar system have rings? Now, calculate the total area of all the rings in the solar system.")


483 73
**Thought**: To answer this question, I need to first identify which planets in the solar system have rings. Then, I can determine the total area of those rings. First, I will search for information about the planets with rings in the solar system.

**Action**: `duckduckgo_search: planets in solar system with rings`  
**Pause**
578 61
**Thought**: To answer this question, I need to determine which planets in the solar system have rings and then calculate the total area of those rings. First, I'll identify the planets with rings.

**Action**: `duckduckgo_search: planets in solar system with rings`  
**Pause**
654 42
**Thought**: I need to search for which planets in the solar system have rings.

**Action**: `duckduckgo_search: how many planets have rings in the solar system`  
**Pause**
752 64
**Thought**: I still need to determine which planets have rings and then gather information about their respective ring areas to calculate the total area. Let me first find out which planet